In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from collections import Counter
import math
import datetime as datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import statistics
import glob
import ipyplot
import numpy as np

/home/jliao/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/jliao/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
filename = 'data/fulldata_20160101_to_20191231.csv'
data = pd.read_csv(filename)
data['datetime_str'] = data.loc[:,'TIME'].apply(datetime.datetime.utcfromtimestamp)

#data.columns

In [3]:
df = pd.DataFrame(data=pd.concat([data['EN_PARA'] ,data['EN_ANTI']],ignore_index = True), columns = ['energy'])

df['pitch angle'] = pd.concat([data['PA_PARA'] ,data['PA_ANTI']],ignore_index = True)
df['flag'] = pd.concat([data['FLAG_PARA'] , -data['FLAG_ANTI']],ignore_index = True)
df['flux'] = pd.concat([data['FLUX_PARA'] ,data['FLUX_ANTI']],ignore_index = True)
df['eflux'] = pd.concat([data['EFLUX_PARA'] ,data['EFLUX_ANTI']],ignore_index = True)
df['bx'] = pd.concat([data['BX_GSM'] ,data['BX_GSM']],ignore_index = True)
df['dist'] = pd.concat([data['DIST'] ,data['DIST']],ignore_index = True)
df['xgsm'] = pd.concat([data['GSM_X'] ,data['GSM_X']],ignore_index = True)
df['ygsm'] = pd.concat([data['GSM_Y'] ,data['GSM_Y']],ignore_index = True)
df['zgsm'] = pd.concat([data['GSM_Z'] ,data['GSM_Z']],ignore_index = True)
df['beta'] = pd.concat([data['BETA'] ,data['BETA']],ignore_index = True)
df['imfBy'] = pd.concat([data['IMF_BY_PARA'] ,data['IMF_BY_ANTI']],ignore_index = True)
df['imfBz'] = pd.concat([data['IMF_BZ_PARA'] ,data['IMF_BZ_ANTI']],ignore_index = True)
df['storm_phase'] = pd.concat([data['STORM_PHASE'] ,data['STORM_PHASE']],ignore_index = True)
df['velocity_o_all'] = pd.concat([data['O_V'] ,data['O_V']],ignore_index = True)
df['v_par_all'] = pd.concat([data['O_VPAR'] ,data['O_VPAR']],ignore_index = True)
df['v_perp_all'] = pd.concat([data['O_VPERP'] ,data['O_VPERP']],ignore_index = True)
df['density_o_all'] = pd.concat([data['O_N'] ,data['O_N']],ignore_index = True)
df['pressure_o_all'] = pd.concat([data['O_P'] ,data['O_P']],ignore_index = True)
df['density_h_all'] = pd.concat([data['H_N'] ,data['H_N']],ignore_index = True)
df['swp'] = pd.concat([data['SW_P_PARA'] ,data['SW_P_ANTI']],ignore_index = True)
df['swv'] = pd.concat([data['SW_V_PARA'] ,data['SW_V_ANTI']],ignore_index = True)
df['kp'] = pd.concat([data['KP'] ,data['KP']],ignore_index = True)
df['dst'] = pd.concat([data['DST'] ,data['DST']],ignore_index = True)
df['datetime_str'] = pd.concat([data['datetime_str'] ,data['datetime_str']],ignore_index = True)
df['year'] = df['datetime_str'].dt.to_period('Y')

df['kp_gt_2'] = df['kp'] > 2 
df['storm'] = df['storm_phase'] > 0
df['storm_phase'] = pd.Categorical(df['storm_phase']).rename_categories({0:'nonstorm',1:'prestorm',2:'main phase',3:'fast recovery', 4:'long recovery'})
df['region'] = df['beta']
df['energy_int'] = round(df['energy'])

df.loc[df['beta'] > 0 , 'region'] = 1
df.loc[df['beta'] > 0.5 , 'region'] = 2
df.loc[df['beta'] > 1 , 'region'] = 3

index2016 = (df['year'] == '2016')
index2017 = (df['year'] == '2017')
index2018 = (df['year'] == '2018')
index2019 = (df['year'] == '2019')

In [9]:
df['flag'].value_counts()

 0.0    559510
 1.0    115590
-1.0    108584
Name: flag, dtype: int64

In [ ]:
print(statistics.median(abs(df['kp'])))
print(statistics.median(abs(df['bx'])))
print(statistics.median(abs(df['v_par_all'])))
print(statistics.median(abs(df['v_perp_all'])))
print(statistics.median(abs(df['swv'])))
print(statistics.median(abs(df['swp'])))

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Kp 2016", "Kp 2017", "Kp 2018", "Kp 2019"))

fig.add_trace(go.Histogram(x=df.loc[index2016, 'kp'], xbins=dict(start=0,end=7, size=1)), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2017, 'kp'], xbins=dict(start=0,end=7, size=1)),row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index2018, 'kp'], xbins=dict(start=0,end=7, size=1)) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2019, 'kp'], xbins=dict(start=0,end=7, size=1)) ,row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("bx 2016", "bx 2017", "bx 2018", "bx 2019"))

fig.add_trace(go.Histogram(x=abs(df.loc[index2016, 'bx']), xbins=dict(start=0,end=100, size=1)), row=1, col=1)
fig.add_trace(go.Histogram(x=abs(df.loc[index2017, 'bx']), xbins=dict(start=0,end=100, size=1)),row=1, col=2)
fig.add_trace(go.Histogram(x=abs(df.loc[index2018, 'bx']), xbins=dict(start=0,end=100, size=1)) ,row=2, col=1)
fig.add_trace(go.Histogram(x=abs(df.loc[index2019, 'bx']), xbins=dict(start=0,end=100, size=1)) ,row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("swv 2016", "swv 2017", "swv 2018", "swv 2019"))

fig.add_trace(go.Histogram(x=df.loc[index2016, 'swv'], xbins=dict(start=0,end=900, size=10)), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2017, 'swv'], xbins=dict(start=0,end=900, size=10)) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index2018, 'swv'], xbins=dict(start=0,end=900, size=10)) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2019, 'swv'], xbins=dict(start=0,end=900, size=10)) ,row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("swp 2016", "swp 2017", "swp 2018", "swp 2019"))

fig.add_trace(go.Histogram(x=df.loc[index2016, 'swp'], xbins=dict(start=0,end=20, size=1)), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2017, 'swp'], xbins=dict(start=0,end=20, size=1)) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index2018, 'swp'], xbins=dict(start=0,end=20, size=1)) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2019, 'swp'], xbins=dict(start=0,end=20, size=1)) ,row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("v_perp_all 2016", "v_perp_all 2017", "v_perp_all 2018", "v_perp_all 2019"))

fig.add_trace(go.Histogram(x=df.loc[index2016, 'v_perp_all'], xbins=dict(start=0,end=500, size=10 )), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2017, 'v_perp_all'], xbins=dict(start=0,end=500, size=10 )) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index2018, 'v_perp_all'], xbins=dict(start=0,end=500, size=10 )) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2019, 'v_perp_all'], xbins=dict(start=0,end=500, size=10 )) ,row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:

fig = make_subplots(rows=2, cols=2, subplot_titles=("storm_phase 2016", "storm_phase 2017", "storm_phase 2018", "storm_phase 2019"))
fig.add_trace(go.Histogram(x=df.loc[index2016, 'storm_phase']), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2017, 'storm_phase']) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index2018, 'storm_phase']) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2019, 'storm_phase']) ,row=2, col=2)

fig.update_xaxes(categoryorder='array', categoryarray= ['nonstorm','prestorm','main phase','fast recovery','long recovery'], row=1, col=1)
fig.update_xaxes(categoryorder='array', categoryarray= ['nonstorm','prestorm','main phase','fast recovery','long recovery'], row=1, col=2)
fig.update_xaxes(categoryorder='array', categoryarray= ['nonstorm','prestorm','main phase','fast recovery','long recovery'], row=2, col=1)
fig.update_xaxes(categoryorder='array', categoryarray= ['nonstorm','prestorm','main phase','fast recovery','long recovery'], row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Storm Phases", "Kp Index", "IMF Bz", "Solar wind Pressure"))

fig.add_trace(go.Histogram(x=df['storm_phase']), row=1, col=1)
fig.add_trace(go.Histogram(x=df['kp'], xbins=dict(start=0,end=7.0, size=0.1 )) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df['imfBz']) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df['swp']) ,row=2, col=2)

fig.update_xaxes(categoryorder='array', categoryarray= ['nonstorm','prestorm','main phase','fast recovery','long recovery'], row=1, col=1)

fig.update_layout(showlegend=False)


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Energy", "Flux", "Pitch Angle", "Region"))

fig.add_trace(go.Histogram(x=df['energy']), row=1, col=1)
fig.add_trace(go.Histogram(x=df['flux'], xbins=dict(start=0,end=200, size=1)) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df['pitch angle']) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df['region'], xbins=dict(start=0,end=4, size=1)) ,row=2, col=2)

#fig.update_xaxes(type="log", range = [0,5],row=1, col=1)
fig.update_xaxes(type="log", range = [0,2],row=1, col=2)
fig.update_xaxes(range = [1,4],row=2, col=2)

#fig.update_xaxes(type="log", row=1, col=2)
#fig.update_xaxes(type="log", row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Vel_tot", "Density", "Pressure", "H+ density"))

fig.add_trace(go.Histogram(x=df['velocity_o_all']), row=1, col=1)
fig.add_trace(go.Histogram(x=df['density_o_all'], xbins=dict(start=0.01,end=10, size=0.1)) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df['pressure_o_all'], xbins=dict(start=0.001,end=0.01, size=0.001)) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df['density_h_all'], xbins=dict(start=0.01,end=10, size=0.1 )) ,row=2, col=2)

#fig.update_xaxes(type="log", range = [0,5],row=1, col=1)
fig.update_xaxes(type="log", range = [-1,1],row=1, col=2)
#fig.update_xaxes(type="log", range = [-4,-2],row=2, col=1)
fig.update_xaxes(type="log", range = [-1,1],row=2, col=2)

#fig.update_xaxes(type="log", row=1, col=2)
#fig.update_xaxes(type="log", row=2, col=2)

fig.update_layout(showlegend=False)
